## Credit to Francesco Crecchi

# MNIST classification using a Convolutional Neural Network (CNN)

In [20]:
# Utility libraries
import os
import random
from datetime import datetime

# Core libraries
import numpy as np
import tensorflow as tf

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Softmax, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras import regularizers
from tensorflow.keras import metrics
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from keras.utils import np_utils

import matplotlib.pyplot as plt # For plotting purposes


# Bug still not solved, resort to old keras module (see below)
# from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

# Use the above import when the bug is solved
import keras
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
# ------------------------------------------#

In [32]:
# Load the TensorBoard notebook extension (for visualization purposes)
%load_ext tensorboard

## Data loading

In [12]:
# Loads the training and test data sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
num_classes = 10

In [13]:
# We reshape the training and test data sets to be a 4D tensor.
# Dimensions: num_images x 28 x 28 x no_channels (e.g. RGB)
# The 1 is because we have a single channel (greyscale). If RGB color images, we'd have 3 channels.
# -1 means "let the library infer the dimension"
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32')
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32')
input_shape = (28, 28, 1)

## Preprocess the data (scale all values between 0 and 1)

### Note: you MUST NOT use statistics coming from the TEST set. That is "cheating"!

In [14]:
# Scales the training and test data to range between 0 and 1.
max_value = X_train.max()
X_train /= max_value
X_test /= max_value

In [15]:
# The training and test labels are integers from 0 to 9 indicating the class label
(y_train, y_test)

(array([5, 0, 4, ..., 5, 6, 8], dtype=uint8),
 array([7, 2, 1, ..., 4, 5, 6], dtype=uint8))

In [16]:
# We convert the class labels to binary class matrices
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)
(y_train, y_test)

(array([[0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32))

## Define a Convolutional Neural Net (ConvNet) with the Sequential API

<img src="https://drive.google.com/uc?id=1rAe5QRDGUstZMrPirJS0hNHf3E8x_05S" width=90%>

In [25]:
model = Sequential()

# Add a convolutional layer. Outputs 10 different "filtered" images (using 10 different 5x5 kernels)
model.add(Conv2D(10, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))

# Reduce the dimension of the filtered images by 2
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add a convolutional layer. Outputs 20 different filters (using 20 different 5x5 kernels)
model.add(Conv2D(20, (5, 5), activation='relu'))

# Reduce the dimension of the filtered images by 2
model.add(MaxPooling2D(pool_size=(2, 2)))

# Dropout is a technique that regularizes the model, by randomly "turning off" a percentage of the neurons
# Here, we randomly shut down 75% of the neurons at each training step
model.add(Dropout(0.25))

# Convert the Dropout output in vectorial form
model.add(Flatten())

# Apply an MLP to the vectorial representation
model.add(Dense(100, activation='relu'))

# Output class probability scores
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 24, 24, 10)        260       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 12, 12, 10)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 20)          5020      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 4, 4, 20)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 4, 20)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 320)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)              

# Have you noticed the number of parameters?? 40k of CNN vs 200K of an MLP!!!
## (32K only for final MLP)
## Moreover, the CNN architecture extracts *local* patches of an image, whereas an MLP combines all pixels together!

## Train Classifier (we won't do model selection today, do it as an exercise)

In [34]:
# Clear any logs from previous runs
!rm -rf ./logs_linear

# Set up a log folder in which we will store the output to be displayed on TensorBoard
logdir = os.path.abspath("logs_cnn/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=logdir)

# Trains the model, iterating on the training data in batches of 128 in 5 epochs.
# Using the SGD optimizer.
model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# We will automatically create a validation split (less flexible for model selection)
model.fit(X_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_split=0.25,
          callbacks=[tensorboard_callback])

Train on 45000 samples, validate on 15000 samples
Epoch 1/10
45000/45000 [==============================] - 6s 142us/sample - loss: 0.1146 - accuracy: 0.9644 - val_loss: 0.0833 - val_accuracy: 0.9745
Epoch 2/10
45000/45000 [==============================] - 6s 134us/sample - loss: 0.1110 - accuracy: 0.9653 - val_loss: 0.0807 - val_accuracy: 0.9755
Epoch 3/10
45000/45000 [==============================] - 6s 132us/sample - loss: 0.1058 - accuracy: 0.9678 - val_loss: 0.0796 - val_accuracy: 0.9755
Epoch 4/10
45000/45000 [==============================] - 6s 131us/sample - loss: 0.1019 - accuracy: 0.9688 - val_loss: 0.0764 - val_accuracy: 0.9771
Epoch 5/10
45000/45000 [==============================] - 6s 134us/sample - loss: 0.0998 - accuracy: 0.9691 - val_loss: 0.0743 - val_accuracy: 0.9783
Epoch 6/10
45000/45000 [==============================] - 6s 133us/sample - loss: 0.0962 - accuracy: 0.9699 - val_loss: 0.0731 - val_accuracy: 0.9784
Epoch 7/10
45000/45000 [==========================

In [35]:
# Visualize your model
%tensorboard --logdir {logdir}

## Model Evaluation

In [37]:
loss, acc = model.evaluate(X_test, y_test)
print("Test set accuracy: {0:.4f}".format(acc))

10000/10000 [==============================] - 1s 79us/sample - loss: 0.0549 - accuracy: 0.9829
Test set accuracy: 0.9829


---

# Using a pre-trained model

Achieving state-of-the-art image recognition performances training a neural network model from scratch is a **very hard** task. Luckly enough, there is no need to reinvent the wheel!
We can simply stand on the giant's shoulder by using some **pre-trained** deep neural network model in our application for image recognition or use it as an extremely powerful features extractor leaving us with just the duty of *specialize* this general-purpose model for our task at-hand!

Let's pretend you are working for a smart/IoT/intelligent/etc. pets feeding bowl startup and your job is to make the bowl able to recognize cats or dogs to provide the right type of food. In other words, you need to recognize a dog from a cat the most accurate as possible. How would you do that?

## Standing on the giant's shoulders!

We can _stand on the giant's shoulders_ by using a state-of-the-art convolutional model as features extractor and by _fine-tune_ the last layers to perform the _cat-vs-dog_ classification task.

We will use the _ResNet-50_ architecture, pre-trained on the ImageNet dataset. Because the ImageNet dataset contains several "cat" classes (persian cat, siamese cat...) and many "dog" classes among its total of 1000 classes, this model will already have learned features that are relevant to our classification problem. In fact, it is possible that merely recording the softmax predictions of the model over our data rather than the bottleneck features would be enough to solve our dogs vs. cats classification problem extremely well. However, the method we present here is more likely to generalize well to a broader range of problems, including problems featuring classes absent from ImageNet.

Here's what the ResNet architecture looks like:

<img src="https://drive.google.com/uc?id=1AminLP0xNTlbbQ8FFKMOGRSw5oeisGqC">

Our strategy will be as follow: we will use the pre-trained convolutional part of the model as a _very_ powerful features extractor for the simple feed-forward binary classifier for distinguishing between cats and dogs images.

# Convolutional Autoencoder for Image Segmentation

![FCN](https://missinglink.ai/wp-content/uploads/2019/03/SegNet-neural-network_2x.png)

### Taken from [here](https://missinglink.ai/guides/computer-vision/image-segmentation-deep-learning-methods-applications/)

# Exercise: Convolutional Auto-Encoders for Anomaly Detection